In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
from colossus.cosmology import cosmology as ccosmology
from scipy.interpolate import UnivariateSpline

from SSLimPy.cosmology import halo_model
from SSLimPy.interface import sslimpy
from SSLimPy.utils.utils import linear_interpolate

plt.style.use("fivethirtyeight")

In [ ]:
# colosus results
ccosmo = ccosmology.setCosmology("planck18")

In [ ]:
# SSLimPy results
cosmodict = {
    "h": ccosmo.h,
    "Omegam": ccosmo.Om0,
    "Omegab": ccosmo.Ob0,
    "ns": ccosmo.ns,
    "sigma8": ccosmo.sigma8,
}

A = sslimpy.sslimpy(cosmopars=cosmodict)

In [ ]:
hm = A.fiducialastro.halomodel
b_ss = hm._bias_function.b1(hm.M, 2.83, 1.686)

In [ ]:
plt.semilogx(hm.M, b_ss)
plt.xlim(1e9,1e15)
plt.ylim(-2, 10)

In [ ]:
hm._bias_function.b1([1e9, 1e15]*u.Msun, 2.8333333, 1.686)

In [ ]:
k = nl.k
Dk = 4 * np.pi * (k / 2 / np.pi) ** 3 * nl.matpow(k, 0, tracer="clustering")
t = np.linspace(0, 1, 400)
rR = np.geomspace(1e-4, 1e3, 8)
I = np.empty((*t.shape, *rR.shape))
for ir, Ri in enumerate(rR):
    I = nonlinear.sigma_integrand(t, Ri, k, Dk, 3)
    plt.semilogy(t, I, label="R={:1.0e}".format(Ri))
plt.legend(loc=8, ncol=4)

In [ ]:
cosmodict = {
    "h": ccosmo.h,
    "Omega_m": ccosmo.Om0,
    "Omega_b": ccosmo.Ob0,
    "n_s": ccosmo.ns,
    "sigma8": ccosmo.sigma8,
    "N_ncdm": 1,
    "m_ncdm": 0.06,
    "N_ur": 2.0308,
}

cosmo = Class()
cosmo.set_default()
cosmo.set({"non linear": "hmcode2020"})
cosmo.set(cosmodict)
cosmo.compute()
class_sigma = np.zeros(len(nl.R))
for iR, Ri in enumerate(nl.R.value):
    class_sigma[iR] = cosmo.sigma_cb(Ri, 0)

In [ ]:
plt.rcParams.keys()

In [ ]:
plt.loglog(nl.R, nl.sigmaR_lut[:, 0], label="New integration method")
plt.loglog(nl.R, class_sigma, label="Class")
plt.loglog(nl.R, nl.sigmaR_of_z(nl.R, 0), label="Old integration method")
Rh = np.geomspace(1e-4, 1e3)
plt.loglog(Rh / ccosmo.h, ccosmo.sigma(Rh, 0), label="Colosus", ls="--")
plt.xlabel(r"$R \:[\mathrm{Mpc}]$")
plt.ylabel(r"$\sigma(R)$")
plt.legend()

In [ ]:
neff_of_R_lut = -2.0 * nl.R.value * nl.dsigmaR_lut[:,0] / nl.sigmaR_lut[:,0] - 3
#extrapolate look up tables to higher R
R = np.geomspace(1e-4, 1e5)
sigma_of_R = np.exp(linear_interpolate(np.log(nl.R.value), np.log(nl.sigmaR_lut[:,0]), np.log(R)))
dsigma_of_R = -np.exp(linear_interpolate(np.log(nl.R.value), np.log(-nl.dsigmaR_lut[:,0]), np.log(R)))

neff_of_R = -2.0 * R * dsigma_of_R / sigma_of_R - 3
plt.semilogx(R, neff_of_R)
plt.semilogx(nl.R, neff_of_R_lut)
plt.hlines(ccosmo.ns-4,1e-4,1e5, color="yellow")
plt.hlines(ccosmo.ns,1e-4,1e5, color="yellow")

In [ ]:
x = np.log((nl.R/(1.5e3*u.Mpc)).to(1).value)
dx = np.diff(x)[0]

f = neff_of_R_lut

f0 = f[-1]
finf = ccosmo.ns
# df0 = (-2 * f[-4] + 9 * f[-3] - 18 * f[-2] + 11 * f[-11]) / (6 * dx)
# ddf0 = (-1 * f[-4] + 4 * f[-3] - 5 * f[-2] + 2 * f[-11]) / (dx**2)
df0 = (1 * f[-3] - 4 * f[-2] + 3 * f[-1]) / (2* dx)
ddf0 = (1 * f[-3] - 2 * f[-2] + 1 * f[-1]) / (dx**2)


d = finf
c = f0
b = df0 + f0 - finf
a = (ddf0 + 2*df0 + f0 - finf) / 2

Rexp = np.geomspace(1.5e3, 1e5)*u.Mpc
xexp = np.log((Rexp/(1.5e3*u.Mpc)).to(1).value)
fexp = (a*xexp**2 + b* xexp + c)*np.exp(-xexp) + d * (1 - np.exp(-xexp))

In [ ]:
plt.semilogx(R, neff_of_R)
plt.semilogx(nl.R, neff_of_R_lut)
plt.semilogx(Rexp, fexp, ls="--", color="black")
plt.hlines(ccosmo.ns-4,1e-4,1e5, color="yellow")
plt.hlines(ccosmo.ns,1e-4,1e5, color="yellow")

In [ ]:
R = np.geomspace(1e-5, 1e4)*u.Mpc
d = nl.read_lut(R, 0.0, "both")

In [ ]:
plt.loglog(R, d["sigma"], label=r"$\sigma$")
plt.loglog(R, -R[:,None] * d["dsigma"]/d["sigma"], label=r"$-\mathrm{d}\log\sigma/\mathrm{d}\log R$")
plt.legend()
plt.xlabel(r"$R \:[\mathrm{Mpc}]$")
plt.ylabel(r"Variance Functions")

In [ ]:
astro = A.curent_astro
rhoM = astro.rho_crit * astro.cosmology.Omega(0, "clustering")
M = 4.0 / 3.0 * np.pi * rhoM * R**3
sM = astro.sigmaM(M, 0, astro.astrotracer)
dsM = astro.dsigmaM_dM(M, 0.0, astro.astrotracer)

In [ ]:
plt.loglog(M, sM)
plt.loglog(M, - M * dsM/sM)

In [ ]:
hmf = astro.bias_coevolution.sc_hmf(M, 0.0, 1.6864)

In [ ]:
plt.loglog(M, M[:,None]**3 * hmf)